In [1]:
#adatletöltés a ˛/data-ba

In [2]:
import pandas as pd

df = pd.read_csv("../data/vehicle_updates.csv")
stop_times = pd.read_csv("../data/stop_times.txt")
trips = pd.read_csv("../data/trips.txt")
df.head()
stop_times.head()
trips.head()
df = df.rename(columns={"delay_seconds_calc": "delay_seconds","stop_id":"last_stop_id"})
print(df)

/tmp/ipykernel_470/3586450070.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/vehicle_updates.csv")
/tmp/ipykernel_470/3586450070.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_times = pd.read_csv("../data/stop_times.txt")
/tmp/ipykernel_470/3586450070.py:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("../data/trips.txt")


               timestamp_utc                vehicle_id       trip_id route_id  \
0        2025-11-10 03:17:07   VehiclePosition-BKK_175       D020714     2660   
1        2025-11-10 03:17:07  VehiclePosition-BKK_2004     D01611294     3060   
2        2025-11-10 03:17:07  VehiclePosition-BKK_2005  D01611118738     3060   
3        2025-11-10 03:17:07  VehiclePosition-BKK_2021  D01611118909     3060   
4        2025-11-10 03:17:07  VehiclePosition-BKK_2022   D0161158930     3040   
...                      ...                       ...           ...      ...   
6378392  2025-11-24 23:11:37  VehiclePosition-BKK_7218    C944381470     1980   
6378393  2025-11-24 23:12:38  VehiclePosition-BKK_7218    C944381470     1980   
6378394  2025-11-24 23:13:38  VehiclePosition-BKK_7469       D047795     2005   
6378395  2025-11-24 23:14:38  VehiclePosition-BKK_7469       D047795     2005   
6378396  2025-11-24 23:15:39  VehiclePosition-BKK_7469       D047795     2005   

               lat        l

In [7]:
end_stops = (
    stop_times
    .sort_values(["trip_id", "stop_sequence"])
    .groupby("trip_id")
    .last()
    .reset_index()[["trip_id", "stop_id"]]
    .rename(columns={"stop_id": "end_stop_id"})
)

print(end_stops)

           trip_id end_stop_id
0         C0418310      F04526
1         C0418311      F04181
2         C0418312      F04526
3         C0418313      F04181
4         C0418314      F04526
...            ...         ...
243425   H55081_43    19836301
243426   H55125_43    19836301
243427   H55176_44    19795278
243428  H80001_237     9134218
243429   H800027_1     9142223

[243430 rows x 2 columns]


In [8]:
veh = df.merge(end_stops, on="trip_id", how="left")
veh[["trip_id", "last_stop_id", "end_stop_id"]].head()

,trip_id,last_stop_id,end_stop_id
0,D020714,022005,F03665
1,D01611294,F00308,009220
2,D01611118738,009220,009220
3,D01611118909,F02476,F02476
4,D0161158930,F01378,F01998


In [9]:
veh = veh.sort_values(["vehicle_id", "vehicle_timestamp"])

veh["run_id"] = (
    (veh["trip_id"] != veh["trip_id"].shift()) |
    (veh["vehicle_id"] != veh["vehicle_id"].shift())
).cumsum()
print(veh)

               timestamp_utc                     vehicle_id    trip_id  \
13022    2025-11-11 21:48:07        VehiclePosition-BKK_100  C89348349   
1008046  2025-11-15 08:48:05        VehiclePosition-BKK_100   D0343490   
1008555  2025-11-15 08:49:05        VehiclePosition-BKK_100   D0343490   
1009066  2025-11-15 08:50:06        VehiclePosition-BKK_100   D0343490   
1009572  2025-11-15 08:51:06        VehiclePosition-BKK_100   D0343490   
...                      ...                            ...        ...   
2305879  2025-11-17 16:28:01  VehiclePosition-BKK_H10636_38  H10636_38   
2306605  2025-11-17 16:29:01  VehiclePosition-BKK_H10636_38  H10636_38   
2307321  2025-11-17 16:30:02  VehiclePosition-BKK_H10636_38  H10636_38   
2308040  2025-11-17 16:31:02  VehiclePosition-BKK_H10636_38  H10636_38   
2308768  2025-11-17 16:32:03  VehiclePosition-BKK_H10636_38  H10636_38   

        route_id        lat        lon      speed  heading  \
13022       2511  47.428417  19.038237  12.346656

In [10]:
end_delay = (
    veh[veh["last_stop_id"] == veh["end_stop_id"]]
    .groupby("run_id")["delay_seconds"]
    .first()
    .reset_index()
    .rename(columns={"delay_seconds": "y_end_delay"})
)
veh = veh.merge(end_delay, on="run_id", how="left")
veh = veh.dropna(subset=["y_end_delay"])
print(veh)

               timestamp_utc                     vehicle_id    trip_id  \
1        2025-11-15 08:48:05        VehiclePosition-BKK_100   D0343490   
2        2025-11-15 08:49:05        VehiclePosition-BKK_100   D0343490   
3        2025-11-15 08:50:06        VehiclePosition-BKK_100   D0343490   
4        2025-11-15 08:51:06        VehiclePosition-BKK_100   D0343490   
5        2025-11-15 08:52:06        VehiclePosition-BKK_100   D0343490   
...                      ...                            ...        ...   
6378392  2025-11-17 16:28:01  VehiclePosition-BKK_H10636_38  H10636_38   
6378393  2025-11-17 16:29:01  VehiclePosition-BKK_H10636_38  H10636_38   
6378394  2025-11-17 16:30:02  VehiclePosition-BKK_H10636_38  H10636_38   
6378395  2025-11-17 16:31:02  VehiclePosition-BKK_H10636_38  H10636_38   
6378396  2025-11-17 16:32:03  VehiclePosition-BKK_H10636_38  H10636_38   

        route_id        lat        lon      speed  heading  \
1           2120  47.480469  19.066154   3.086664

In [13]:
import pandas as pd

# megnézzük, mik a duplikált oszlopnevek
dups = pd.Series(veh.columns)[pd.Series(veh.columns).duplicated(keep=False)]
print("Duplikált oszlopnevek:", dups.unique())

# a MÁSODIK / UTOLSÓ előfordulás marad meg
# duplikált oszlopok törlése úgy, hogy a MÁSODIK/UTOLSÓ maradjon meg
veh = veh.loc[:, ~veh.columns.duplicated(keep="last")]


print("Oszlopok a tisztítás után:")
print(veh.columns.tolist())


Duplikált oszlopnevek: ['delay_seconds' 'y_end_delay']
Oszlopok a tisztítás után:
['timestamp_utc', 'vehicle_id', 'trip_id', 'route_id', 'lat', 'lon', 'speed', 'heading', 'vehicle_timestamp', 'last_stop_id', 'current_stop_sequence', 'arrival_time', 'scheduled_arrival', 'delay_seconds', 'end_stop_id', 'run_id', 'y_end_delay']


In [14]:
veh = veh.rename(columns={
    "delay_seconds": "delay_seconds_calc",
    "y_end_delay": "y_end_delay_calc"
})

In [19]:
veh["timestamp_utc"] = pd.to_datetime(veh["timestamp_utc"])
veh["vehicle_timestamp"] = pd.to_datetime(veh["vehicle_timestamp"])
veh["scheduled_arrival"] = pd.to_datetime(veh["scheduled_arrival"])
veh["hour"] = veh["vehicle_timestamp"].dt.hour
veh["weekday"] = veh["vehicle_timestamp"].dt.weekday
print(veh)

              timestamp_utc                     vehicle_id    trip_id  \
1       2025-11-15 08:48:05        VehiclePosition-BKK_100   D0343490   
2       2025-11-15 08:49:05        VehiclePosition-BKK_100   D0343490   
3       2025-11-15 08:50:06        VehiclePosition-BKK_100   D0343490   
4       2025-11-15 08:51:06        VehiclePosition-BKK_100   D0343490   
5       2025-11-15 08:52:06        VehiclePosition-BKK_100   D0343490   
...                     ...                            ...        ...   
6378392 2025-11-17 16:28:01  VehiclePosition-BKK_H10636_38  H10636_38   
6378393 2025-11-17 16:29:01  VehiclePosition-BKK_H10636_38  H10636_38   
6378394 2025-11-17 16:30:02  VehiclePosition-BKK_H10636_38  H10636_38   
6378395 2025-11-17 16:31:02  VehiclePosition-BKK_H10636_38  H10636_38   
6378396 2025-11-17 16:32:03  VehiclePosition-BKK_H10636_38  H10636_38   

        route_id        lat        lon      speed  heading  \
1           2120  47.480469  19.066154   3.086664    356.0   

In [26]:
train_df = veh[[
    "timestamp_utc",
    "vehicle_timestamp",
    "vehicle_id",
    "trip_id",
    "route_id",
    "last_stop_id",
    "end_stop_id",
    "current_stop_sequence",
    "hour",
    "weekday",
    "delay_seconds_calc",
    "y_end_delay_calc",
]].dropna()


In [27]:
train_df.to_csv("../data/processed_dataset.csv", index=False)
print("Saved:", len(train_df), "rows")


Saved: 6170467 rows
